## Simulate data and parameter recovery

Given that we are interested in the drift rate $v$, let see how which model recovers the drift rate $v$ best.

**Note: this part hasn't been finished yet**


### First Generate random day with certain drift rate

In [ ]:
# check which python is in use.
import sys
print('Notebook is running:', sys.executable)

# further check your python version
from platform import python_version

print('The current HDDM version is', python_version())

# If you are sure that conda is installed, also check the package that install
#!conda list  # list the conda

import hddm, IPython
print('The current HDDM version is', hddm.__version__) # 0.8.0

# Warning:`IPython.parallel` package has been deprecated since IPython 4.0. 
print('The current IPython version is', IPython.__version__) 

In [ ]:
# Generate random data, assuming that only drift rate is different. 
# for more details about hddm.generate.gen_rand(), uncomment below:
# ?hddm.generate.gen_rand
# import kabuki
# ?kabuki.generate.gen_rand_data

df_gen, prms_gen = hddm.generate.gen_rand_data({'cond1': {'v':1.5, 'a':1, 't':.4, 'z': 0.55},
                                                'cond2': {'v':2.5, 'a':1, 't':.4, 'z': 0.55},
                                                'cond3': {'v':3.5, 'a':1, 't':.4, 'z': 0.55}}, 
                                               n_fast_outliers= 5, 
                                               n_slow_outliers=5,
                                               subj_noise=0.2, # how much to pertub individual subj params, default = 0.1
                                               size = 300,
                                               subjs = 30)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

g = sns.FacetGrid(df_gen, col='subj_idx', col_wrap=6, hue="condition", height=4, palette="bright")
g = (g.map(sns.distplot, "rt", hist = False, kde = True,
           kde_kws = {'shade': True, 'linewidth': 2}, rug=True)
     .set(xlim=(0, 1.5))  # , ylim=(0, 12))
     .add_legend())

In [ ]:
# print generated params
prms_gen

### Parameter recovery

I.e., Fit the model with the generated data

In [ ]:
## Recover model 1, only free parameter v

rcv_m1 = hddm.HDDM(df_gen, depends_on={'v':['condition']},p_outlier=0.05)

rcv_m1.find_starting_values()
rcv_m1.sample(3000, burn=1500, dbname='rcv_m1.db', db='pickle')
rcv_m1.save('rcv_m1')

In [ ]:
rcv_m1.plot_posteriors(['a', 't', 'v', 'a_std'])

In [ ]:
# rcv_m1.plot_posterior_predictive()

In [ ]:
v_cond1, v_cond2, v_cond3 = rcv_m1.nodes_db.node[['v(cond1)', 'v(cond2)', 'v(cond3)']]
hddm.analyze.plot_posterior_nodes([v_cond1, v_cond2, v_cond3])
plt.xlabel('drift-rate')
plt.ylabel('Posterior probability')
plt.title('Posterior of drift-rate group means')
#plt.savefig('hddm_demo_fig_06.pdf')

In [ ]:
rcv_m2 = hddm.HDDM(df_gen, depends_on={'v':['condition'], 'a':['condition'], 't':['condition']},
                   p_outlier=0.05)

rcv_m2.find_starting_values()
rcv_m2.sample(3000, burn=1500, dbname='rcv_m2.db', db='pickle')
rcv_m2.save('rcv_m2')

In [ ]:
v_cond1, v_cond2, v_cond3 = rcv_m2.nodes_db.node[['v(cond1)', 'v(cond2)', 'v(cond3)']]
hddm.analyze.plot_posterior_nodes([v_cond1, v_cond2, v_cond3])
plt.xlabel('drift-rate')
plt.ylabel('Posterior probability')
plt.title('Posterior of drift-rate group means')

In [ ]:
rcv_m3 = hddm.HDDM(df_gen, depends_on={'v':['condition'], 'z':['condition']},
                   p_outlier=0.05)

rcv_m3.find_starting_values()
rcv_m3.sample(3000, burn=1500, dbname='rcv_m3.db', db='pickle')
rcv_m3.save('rcv_m3')


In [ ]:
v_cond1, v_cond2, v_cond3 = rcv_m3.nodes_db.node[['v(cond1)', 'v(cond2)', 'v(cond3)']]
hddm.analyze.plot_posterior_nodes([v_cond1, v_cond2, v_cond3])
plt.xlabel('drift-rate')
plt.ylabel('Posterior probability')
plt.title('Posterior of drift-rate group means')

In [ ]:
rcv_m4 = hddm.HDDM(df_gen, depends_on={'v':['condition'], 'z':['condition'], 't':['condition']},
                   p_outlier=0.05)

rcv_m4.find_starting_values()
rcv_m4.sample(3000, burn=1500, dbname='rcv_m4.db', db='pickle')
rcv_m4.save('rcv_m4')

v_cond1, v_cond2, v_cond3 = rcv_m4.nodes_db.node[['v(cond1)', 'v(cond2)', 'v(cond3)']]
hddm.analyze.plot_posterior_nodes([v_cond1, v_cond2, v_cond3])
plt.xlabel('drift-rate')
plt.ylabel('Posterior probability')
plt.title('Posterior of drift-rate group means')

In [ ]:
rcv_m1.nodes_db.node[['v(cond1)', 'v(cond2)', 'v(cond3)']].head(20)

### Compare the recovered parameter with the ground truth

plot the $v$ fro three conditions and compare them with the ground truth: 1.5, 2.5, 3.5